In [1]:
#Les imports ici

import os #Permet d'acceder au csv

import pandas as pd #Lire csv, 

import unicodedata #pour checker comment sont les charactères

import torch 

from PIL import Image #Dans le __getitem__ du dataset pour ouvrir l'image dans le dataset

from torch.utils.data import DataLoader, Dataset #Pour créer les train/test_loader et le dataset lié au image/artistes

import torchvision.transforms as transforms #pour la transformation utiliser du a notre pre-tained model

from torchvision import models #Va nous servir a recup le model resNet50 pré-entrainé

import torch.nn as nn #Va nous servir a remplacer la dernière couche du modèle par le nombre de classe qu'on veut en sorti (le nb d'artiste)

from tqdm import tqdm #Utile pour voir la progression de l'entrainement

import torch.optim as optim

### On essaiera d'affiné le résultat de notre CNN sans se soucié du modèle pré-entrainer utilisé pour le moment.

##### Pour le première essais On utilise la donnée comme tel __SANS data-augmentation__
##### Model pré-entrainer: __resNet50__
##### On utilise uniquement le __nom__ de nos artistes lors de ce premier essais

In [2]:
#Acceder au csv

dir = './art-challenge'
path = os.path.join(dir, 'artists.csv')

#Charger le csv

artists_df = pd.read_csv(path) #df pour data frame

#On va créer un dictionnaire {artist_name : index_associé}

artists = artists_df["name"].unique()

def espaceReplacement(name): #On va remplacer les espaces par _ pour que ça correspondent au nom des file d'image
    return ''.join(c if unicodedata.category(c) != 'Zs' else '_' for c in name)

artists = [espaceReplacement(artist) for artist in artists] #La on a tous les artistes

print(artists)

artist_to_idx = {artist: idx for (idx, artist) in enumerate(artists)} #Voici le dictionnaire


['Amedeo_Modigliani', 'Vasiliy_Kandinskiy', 'Diego_Rivera', 'Claude_Monet', 'Rene_Magritte', 'Salvador_Dali', 'Edouard_Manet', 'Andrei_Rublev', 'Vincent_van_Gogh', 'Gustav_Klimt', 'Hieronymus_Bosch', 'Kazimir_Malevich', 'Mikhail_Vrubel', 'Pablo_Picasso', 'Peter_Paul_Rubens', 'Pierre-Auguste_Renoir', 'Francisco_Goya', 'Frida_Kahlo', 'El_Greco', 'Albrecht_Dürer', 'Alfred_Sisley', 'Pieter_Bruegel', 'Marc_Chagall', 'Giotto_di_Bondone', 'Sandro_Botticelli', 'Caravaggio', 'Leonardo_da_Vinci', 'Diego_Velazquez', 'Henri_Matisse', 'Jan_van_Eyck', 'Edgar_Degas', 'Rembrandt', 'Titian', 'Henri_de_Toulouse-Lautrec', 'Gustave_Courbet', 'Camille_Pissarro', 'William_Turner', 'Edvard_Munch', 'Paul_Cezanne', 'Eugene_Delacroix', 'Henri_Rousseau', 'Georges_Seurat', 'Paul_Klee', 'Piet_Mondrian', 'Joan_Miro', 'Andy_Warhol', 'Paul_Gauguin', 'Raphael', 'Michelangelo', 'Jackson_Pollock']


#### on crée le dataset de toutes les images d'un folder, on leur donnera le label/index associé à l'artiste de l'image.

In [8]:
class ArtDataset(Dataset):
    def __init__(self, artist_to_idx, image_folder, transformation):
        self.artist_to_idx = artist_to_idx
        self.image_folder = image_folder
        self.transformation = transformation
        self.data = {} #dictionnaire { image_path: label/index_associé}
        self.image_path= [] #on veut pouvoir garder les paths pour utiliser l'index dans __getitem__

        for img_file in os.listdir(self.image_folder): #os.listdir(image_folder) liste tout les files dans image_folder 
            artist_name = img_file.rsplit('_',1)[0] #la on divise apres le dernier '_' 
            if artist_name in self.artist_to_idx:
                label = self.artist_to_idx[artist_name]
                img_path = os.path.join(self.image_folder, img_file)
                self.data[img_path] = label
                self.image_path.append(img_path)
            else:
                print(f"Avertissement: '{artist_name}' ne se trouve pas dans ton dictionnaire")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = self.image_path[index]
        label = self.data[img_path]
        image = Image.open(img_path).convert('RGB') #On doit convertir en RGB car certaine image sont en niveau de gris (1 canal) et d'autre en couleur
        if self.transformation:
            image = self.transformation(image)
        return image, label
    

In [9]:
#On va transformer les images, pour avoir des images de tailles uniforme mais aussi qui correspondent au standard utilisé par resNet50

transformation = transforms.Compose([
    transforms.Resize((224, 224)),  # Taille d'entrée ResNet50
    transforms.ToTensor(),  #redim les valeurs de pixels de [0, 255] à [0, 1] et c'est les tenseurs sont les formats requis pour des réseaux de neuronnes Pytorch
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalisation pour ResNet, Ces valeurs sont calculées à partir de la base de données ImageNet (sur laquelle ResNet est pré-entraîné)
])

lq_folder = './art-challenge/images_lq'
dataset = ArtDataset(artist_to_idx, lq_folder , transformation)

train_size = int(len(dataset)*0.7) #On prend 80% pour le training, Attention on peut pas prendre de float!
val_size = int(len(dataset)*0.15)
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset ,test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size ,test_size])
#La c'est des sous-ensemble du dataset initial 

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#Plus besoin de shuffle ici vu que ça fait plus parti de l'entrainement du model
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [10]:
print(dataset[0]) #parfait, l'index 19 correspond bien a Dürer

(tensor([[[0.8789, 0.7933, 0.6563,  ..., 0.4337, 0.4851, 0.4337],
         [0.6563, 0.6221, 0.6221,  ..., 0.1768, 0.2282, 0.2796],
         [0.5193, 0.4508, 0.4337,  ..., 0.1426, 0.1083, 0.2282],
         ...,
         [0.8104, 0.6563, 0.6049,  ..., 0.8276, 0.8104, 0.8447],
         [0.8618, 0.7419, 0.6906,  ..., 0.7933, 0.8104, 0.8789],
         [0.8789, 0.7419, 0.6734,  ..., 0.8104, 0.8789, 0.8961]],

        [[1.0280, 0.9405, 0.8004,  ..., 0.5728, 0.6254, 0.5728],
         [0.8004, 0.7654, 0.7654,  ..., 0.3102, 0.3627, 0.4153],
         [0.6604, 0.5903, 0.5728,  ..., 0.2752, 0.2402, 0.3627],
         ...,
         [0.9580, 0.8004, 0.7479,  ..., 0.9755, 0.9580, 0.9930],
         [1.0105, 0.8880, 0.8354,  ..., 0.9405, 0.9580, 1.0280],
         [1.0280, 0.8880, 0.8179,  ..., 0.9580, 1.0280, 1.0455]],

        [[1.2457, 1.1585, 1.0191,  ..., 0.7925, 0.8448, 0.7925],
         [1.0191, 0.9842, 0.9842,  ..., 0.5311, 0.5834, 0.6356],
         [0.8797, 0.8099, 0.7925,  ..., 0.4962, 0.4614, 0

#### On a récupéré les dataloader, on les utilisera durant la phase d'entrainement et de test

#### Il faut maintenant chargé le modèle resnet50 pré-entrainé et changer la derniere couche pour qu'elle correspondent au nombre d'artiste dans notre csv

In [50]:
model = models.resnet50(weights='DEFAULT') #On charge le modèle

avant_derniere_couche = model.fc.in_features #On recup la taille de l'avant derniere couche
model.fc = nn.Linear(avant_derniere_couche, len(artists)) #On a change la derniere couche

#A partir de la on va commencer a entrainer donc on place le device dans cuda pour utiliser le gpu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device) #On déplace le modèle dedans

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to C:\Users\benad/.cache\torch\hub\checkpoints\resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:21<00:00, 4.67MB/s]


In [11]:
#On regarde si on est bien sur le gpu

torch.cuda.is_available()

True

In [70]:
criterion = nn.CrossEntropyLoss()
optimize = optim.Adam(model.parameters(), lr=0.001)

def training(num_epochs, model, criterion,optimizer):

    for epoch in range(num_epochs):
        model.train()
        total_train_loss= []
        t = tqdm(train_loader)
        for image, label in t:
            image, label = image.to(device), label.to(device)
            outputs = model(image)
            loss = criterion(outputs, label)

            optimizer.zero_grad() #on remet a zero les gradients pour pas que ça s'accumule
            loss.backward()
            optimizer.step()

            total_train_loss.append(loss.item())

        avg_train_loss = sum(total_train_loss)/ len(train_loader)

        model.eval()
        total_val_loss = []
        correct = 0
        with torch.no_grad(): #Permet de ne plus mettre a jour les poids 
            for image, label in val_loader:
                image, label = image.to(device), label.to(device)
                outputs = model(image)
                val_loss = criterion(outputs, label)
                _, predicted = torch.max(outputs,1) #permet d'obtenir quelle classe la plus grande proba, donc quelle a ete la prediction
                correct += (predicted == label).sum().item() #?
                total_val_loss.append(val_loss.item())

        avg_val_loss = sum(total_val_loss)/len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")


    model.eval()
    total_test_loss=[]
    correct_test = 0
    total_test=0
    with torch.no_grad():
        for image,label in test_loader:
            image, label = image.to(device), label.to(device)
            outputs = model(image)
            test_loss = criterion(outputs, label)
            _, predicted_test = torch.max(outputs, 1)
            correct_test += (predicted_test == label).sum().item()
            total_test_loss.append(test_loss.item())
            total_test += label.size(0)

    avg_test_loss = sum(total_test_loss)/len(test_loader)
    accuracy = correct_test/total_test*100
    print(f"Test Loss: {avg_test_loss:.4f}, test Accuracy: {accuracy:.4f} ")
    

In [71]:
training(10, model, criterion, optimize)

100%|██████████| 183/183 [01:53<00:00,  1.61it/s]


Epoch 1/10, Train Loss: 0.1404, Avg Val Loss: 1.4788


100%|██████████| 183/183 [01:52<00:00,  1.62it/s]


Epoch 2/10, Train Loss: 0.0851, Avg Val Loss: 1.8157


100%|██████████| 183/183 [01:56<00:00,  1.57it/s]


...

Epoch 8/10, Train Loss: 0.0521, Avg Val Loss: 1.7221


100%|██████████| 183/183 [01:56<00:00,  1.57it/s]


Epoch 9/10, Train Loss: 0.0712, Avg Val Loss: 1.9549


100%|██████████| 183/183 [01:55<00:00,  1.59it/s]


Epoch 10/10, Train Loss: 0.0597, Avg Val Loss: 1.4783
Test Loss: 1.3721, test Accuracy: 69.3780 


### Les prochaines ameliorations possible sont soit de faire de la data augmentation pour homogénéïsé le nombre d'image pour chaque artiste
### ou bien on peut trouver un moyen d'utiliser les autres données dans notre csv comme la biographie des auteurs.

#### On commence par la 2eme amelioration ( ajouter de nouvelle caractéristique). ON les ajoute directement a notre pipeline ArtDataset

In [41]:
#On va dabord rajouter les données additionelle dans un nouveau dictionnaire

dir = './art-challenge/'
path = os.path.join(dir, 'artists.csv')

artist_df_2 = pd.read_csv(path)

additional_data = artist_df_2.set_index("name").T.to_dict() #utilise la colonne "name" comme index, chq artiste devient un clé
#T.to_dict() transpose le datafram epour que chaque ligne devienne un dictionnaire de donnée par artiste

#On l'affiche pour voir si ça ressemble a ce qu'on veut

print(additional_data)

#OK c'est bon 

{'Amedeo Modigliani': {'id': 0, 'years': '1884 - 1920', 'genre': 'Expressionism', 'nationality': 'Italian', 'bio': "Amedeo Clemente Modigliani (Italian pronunciation: [ameˈdɛːo modiʎˈʎaːni]; 12 July 1884 – 24 January 1920) was an Italian Jewish painter and sculptor who worked mainly in France. He is known for portraits and nudes in a modern style characterized by elongation of faces, necks, and figures that were not received well during his lifetime but later found acceptance. Modigliani spent his youth in Italy, where he studied the art of antiquity and the Renaissance. In 1906 he moved to Paris, where he came into contact with such artists as Pablo Picasso and Constantin Brâncuși. By 1912 Modigliani was exhibiting highly stylized sculptures with Cubists of the Section d'Or group at the Salon d'Automne.", 'wikipedia': 'http://en.wikipedia.org/wiki/Amedeo_Modigliani', 'paintings': 193}, 'Vasiliy Kandinskiy': {'id': 1, 'years': '1866 - 1944', 'genre': 'Expressionism,Abstractionism', 'na

In [42]:
class ArtDatasetv2(Dataset):
    def __init__(self, image_folder, artist_to_idx, transformation, additional_data):
        self.image_folder = image_folder
        self.artist_to_idx = artist_to_idx
        self.transformation = transformation
        self.additional_data = additional_data
        self.data = {}
        self.image_path = []

        for img_file in os.listdir(self.image_folder):
            artist_name = img_file.rsplit('_', 1)[0]
            if artist_name in self.artist_to_idx:
                artist_name_nospace = artist_name.replace('_', ' ')
                label = self.artist_to_idx[artist_name]
                img_path = os.path.join(self.image_folder, img_file)
                additional_info = self.additional_data.get(artist_name_nospace, {})

                # Ajouter toutes les informations supplémentaires à `self.data`
                self.data[img_path] = {
                    "label": label,
                    "years": additional_info.get("years", "Unknown"),
                    "genre": additional_info.get("genre", "Unknown"),
                    "nationality": additional_info.get("nationality", "Unknown"),
                    "bio": additional_info.get("bio", "Unknown"),
                    "wikipedia": additional_info.get("wikipedia", "Unknown"),
                    "paintings": additional_info.get("paintings", "Unknown")
                }
                self.image_path.append(img_path)
            else:
                print(f"Avertissement : '{artist_name}' n'est pas dans artist_to_idx")

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path = self.image_path[index]
        image = Image.open(img_path).convert('RGB')
        
        if self.transformation:
            image = self.transformation(image)
        
        # Récupérer toutes les informations stockées dans `self.data`
        data_info = self.data[img_path]
        label = data_info["label"]
        years = data_info["years"]
        genre = data_info["genre"]
        nationality = data_info["nationality"]
        bio = data_info["bio"]
        wikipedia = data_info["wikipedia"]
        paintings = data_info["paintings"]

        return image, label, years, genre, nationality, bio, wikipedia, paintings


In [43]:
from torch.utils.data import DataLoader

lq_folder = './art-challenge/images_lq'

transformation = transforms.Compose([
    transforms.Resize((224, 224)),  # Taille d'entrée ResNet50
    transforms.ToTensor(),  #redim les valeurs de pixels de [0, 255] à [0, 1] et c'est les tenseurs sont les formats requis pour des réseaux de neuronnes Pytorch
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalisation pour ResNet, Ces valeurs sont calculées à partir de la base de données ImageNet (sur laquelle ResNet est pré-entraîné)
])

dataset = ArtDatasetv2(
    image_folder=lq_folder,
    artist_to_idx=artist_to_idx,
    transformation=transformation,
    additional_data=additional_data
)

# Créez le DataLoader
train_size = int(len(dataset)*0.7) #On prend 80% pour le training, Attention on peut pas prendre de float!
val_size = int(len(dataset)*0.15)
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset ,test_dataset = torch.utils.data.random_split(dataset, [train_size, val_size ,test_size])
#La c'est des sous-ensemble du dataset initial 

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
#Plus besoin de shuffle ici vu que ça fait plus parti de l'entrainement du model
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(dataset[0])

(tensor([[[0.8789, 0.7933, 0.6563,  ..., 0.4337, 0.4851, 0.4337],
         [0.6563, 0.6221, 0.6221,  ..., 0.1768, 0.2282, 0.2796],
         [0.5193, 0.4508, 0.4337,  ..., 0.1426, 0.1083, 0.2282],
         ...,
         [0.8104, 0.6563, 0.6049,  ..., 0.8276, 0.8104, 0.8447],
         [0.8618, 0.7419, 0.6906,  ..., 0.7933, 0.8104, 0.8789],
         [0.8789, 0.7419, 0.6734,  ..., 0.8104, 0.8789, 0.8961]],

        [[1.0280, 0.9405, 0.8004,  ..., 0.5728, 0.6254, 0.5728],
         [0.8004, 0.7654, 0.7654,  ..., 0.3102, 0.3627, 0.4153],
         [0.6604, 0.5903, 0.5728,  ..., 0.2752, 0.2402, 0.3627],
         ...,
         [0.9580, 0.8004, 0.7479,  ..., 0.9755, 0.9580, 0.9930],
         [1.0105, 0.8880, 0.8354,  ..., 0.9405, 0.9580, 1.0280],
         [1.0280, 0.8880, 0.8179,  ..., 0.9580, 1.0280, 1.0455]],

        [[1.2457, 1.1585, 1.0191,  ..., 0.7925, 0.8448, 0.7925],
         [1.0191, 0.9842, 0.9842,  ..., 0.5311, 0.5834, 0.6356],
         [0.8797, 0.8099, 0.7925,  ..., 0.4962, 0.4614, 0

#### Maintenant il faut réussir a intégrer ces nouvelles données au model. Il faut pouvoir traiter les données textuel, 

In [44]:
import torch
import torch.nn as nn
from torchvision import models

class ArtModelv2(nn.Module):
    def __init__(self, num_classes, genre_vocab_size, nationality_vocab_size, years_vocab_size, bio_vocab_size, wikipedia_vocab_size, paintings_vocab_size, embedding_dim=50):
        super(ArtModelv2, self).__init__()
        
        # Modèle pour les images (ex. ResNet50)
        self.image_model = models.resnet50(weights='DEFAULT')
        self.image_model.fc = nn.Identity()  # Retire la dernière couche pour obtenir les features
        
        # Embeddings pour genre et nationalité et les autres info
        self.genre_embedding = nn.Embedding(genre_vocab_size, embedding_dim)
        self.years_embedding = nn.Embedding(years_vocab_size, embedding_dim)
        self.bio_embedding = nn.Embedding(bio_vocab_size, embedding_dim)
        self.wikipedia_embedding = nn.Embedding(wikipedia_vocab_size, embedding_dim)
        self.paintings_embedding = nn.Embedding(paintings_vocab_size, embedding_dim)
        self.nationality_embedding = nn.Embedding(nationality_vocab_size, embedding_dim)
        
        # Couche fully connected pour combiner les features
        self.fc = nn.Linear(2048 + 2 * embedding_dim, num_classes)
    
    def forward(self, image, genre, years, wikipedia, paintings, bio, nationality):
        # Extraire les features d'image
        image_features = self.image_model(image)
        
        # Embeddings pour genre et nationalité
        genre_features = self.genre_embedding(genre)
        years_features = self.years_embedding(years)
        bio_features = self.bio_embedding(bio)
        wikipedia_features = self.wikipedia_embedding(wikipedia)
        paintings_features = self.paintings_embedding(paintings)
        nationality_features = self.nationality_embedding(nationality)
        
        # Combiner les features
        combined_features = torch.cat((image_features, genre_features, years_features,wikipedia_features,bio_features,paintings_features , nationality_features), dim=1)
        
        # Passer par une dernière couche pour prédire la classe
        output = self.fc(combined_features)
        
        return output


In [45]:
# Regroupe toutes les valeurs uniques pour chaque clé
genres = set()
nationalities = set()
years = set()
bios = set()
wikipedias = set()
paintings = set()

for artist_info in additional_data.values():
    genres.add(artist_info.get('genre', 'Unknown'))
    nationalities.add(artist_info.get('nationality', 'Unknown'))
    years.add(artist_info.get('years', 'Unknown'))
    bios.add(artist_info.get('bio', 'Unknown'))
    wikipedias.add(artist_info.get('wikipedia', 'Unknown'))
    paintings.add(artist_info.get('paintings', 'Unknown'))

# Compte les éléments uniques pour chaque champ
genre_vocab_size = len(genres)
nationality_vocab_size = len(nationalities)
years_vocab_size = len(years)
bio_vocab_size = len(bios)
wikipedia_vocab_size = len(wikipedias)
paintings_vocab_size = len(paintings)

print("Genre vocab size:", genre_vocab_size)
print("Nationality vocab size:", nationality_vocab_size)
print("years vocab size:", years_vocab_size)
print("Bio vocab size:", bio_vocab_size)
print("Wikipedia vocab size:", wikipedia_vocab_size)
print("Paintings vocab size:", paintings_vocab_size)


Genre vocab size: 31
Nationality vocab size: 17
years vocab size: 50
Bio vocab size: 50
Wikipedia vocab size: 50
Paintings vocab size: 48


In [46]:
# Création de l'instance du modèle avec les arguments requis
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ArtModelv2(
    num_classes=49,
    genre_vocab_size=genre_vocab_size,
    nationality_vocab_size=nationality_vocab_size,
    years_vocab_size=years_vocab_size,
    bio_vocab_size=bio_vocab_size,
    wikipedia_vocab_size=wikipedia_vocab_size,
    paintings_vocab_size=paintings_vocab_size,
    embedding_dim=50  # Optionnel, sauf si tu souhaites changer la dimension par défaut
).to(device)


In [47]:
from sklearn.preprocessing import LabelEncoder

# Initialisez les encodeurs
genre_encoder = LabelEncoder()
nationality_encoder = LabelEncoder()
years_encoder = LabelEncoder()
wikipedia_encoder = LabelEncoder()
bio_encoder = LabelEncoder()
paintings_encoder = LabelEncoder()

# Entraînez-les sur toutes les valeurs uniques
genres = [info["genre"] for info in additional_data.values()]
nationalities = [info["nationality"] for info in additional_data.values()]
years = [info["years"] for info in additional_data.values()]
wikipedias = [info["wikipedia"] for info in additional_data.values()]
bios = [info["bio"] for info in additional_data.values()]
paintings = [info["paintings"] for info in additional_data.values()]

genre_encoder.fit(genres)
nationality_encoder.fit(nationalities)
years_encoder.fit(years)
wikipedia_encoder.fit(wikipedias)
bio_encoder.fit(bios)
paintings_encoder.fit(paintings)

# Convertissez les genres et nationalités en indices
for artist, info in additional_data.items():
    info["genre"] = genre_encoder.transform([info["genre"]])[0]
    info["nationality"] = nationality_encoder.transform([info["nationality"]])[0]
    info["years"] = years_encoder.transform([info["years"]])[0]
    info["wikipedia"] = wikipedia_encoder.transform([info["wikipedia"]])[0]
    info["bio"] = bio_encoder.transform([info["bio"]])[0]
    info["paintings"] = paintings_encoder.transform([info["paintings"]])[0]


In [50]:
def trainingv2(num_epochs, model, criterion, optimizer):
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = []
        t = tqdm(train_loader)
        for image, label, years, genre, nationality, bio, wikipedia, paintings in t:
            image, label = image.to(device), label.to(device)
            years = years.to(device)
            genre, nationality =genre.to(device), nationality.to(device)
            bio, paintings, wikipedia = bio.to(device), paintings.to(device), wikipedia.to(device)
            
            outputs = model(image, label, years, genre, nationality, bio, wikipedia, paintings)
            loss = criterion(outputs, label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_train_loss.append(loss.item())
        
        avg_train_loss = sum(total_train_loss) / len(train_loader)

        model.eval()
        total_val_loss = []
        correct = 0
        with torch.no_grad(): #Permet de ne plus mettre a jour les poids 
            for image, label, years, genre, nationality, bio, wikipedia, paintings in val_loader:
                image, label = image.to(device), label.to(device)
                years = years.to(device)
                genre, nationality =genre.to(device), nationality.to(device)
                bio, paintings, wikipedia = bio.to(device), paintings.to(device), wikipedia.to(device)
            
                outputs = model(image, label, years, genre, nationality, bio, wikipedia, paintings)
                val_loss = criterion(outputs, label)
                _, predicted = torch.max(outputs,1) #permet d'obtenir quelle classe la plus grande proba, donc quelle a ete la prediction
                correct += (predicted == label).sum().item() #?
                total_val_loss.append(val_loss.item())

        avg_val_loss = sum(total_val_loss)/len(val_loader)

        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Avg Val Loss: {avg_val_loss:.4f}")
    
    model.eval()
    total_test_loss=[]
    correct_test = 0
    total_test=0
    with torch.no_grad():
        for image, label, years, genre, nationality, bio, wikipedia, paintings in test_loader:
            image, label = image.to(device), label.to(device)
            years = years.to(device)
            genre, nationality = genre.to(device), nationality.to(device)
            bio, paintings, wikipedia = bio.to(device), paintings.to(device), wikipedia.to(device)
            
            outputs = model(image, label, years, genre, nationality, bio, wikipedia, paintings)
            test_loss = criterion(outputs, label)
            
            _, predicted_test = torch.max(outputs, 1)
            correct_test += (predicted_test == label).sum().item()
            total_test_loss.append(test_loss.item())
            total_test += label.size(0)

    avg_test_loss = sum(total_test_loss)/len(test_loader)
    accuracy = correct_test/total_test*100
    print(f"Test Loss: {avg_test_loss:.4f}, test Accuracy: {accuracy:.4f} ")
    
        


In [51]:
criterion = nn.CrossEntropyLoss()
optimize = optim.Adam(model.parameters(), lr=0.001)
trainingv2(10, model, criterion, optimize)

  0%|          | 0/183 [00:00<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'to'